In [118]:
import time
import nrrd
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
import h5py

from dipy.tracking.local import LocalTracking, ThresholdTissueClassifier
from dipy.tracking.utils import random_seeds_from_mask
from dipy.reconst.dti import TensorModel
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,
                                   auto_response)
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model
from dipy.data import fetch_stanford_hardi, read_stanford_hardi, get_sphere
from dipy.segment.mask import median_otsu
from dipy.viz import actor, window
from dipy.io.image import save_nifti
from dipy.io import read_bvals_bvecs
from dipy.core import gradients
from dipy.tracking.streamline import Streamlines
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, gradient_table_from_bvals_bvecs
from dipy.reconst.dti import fractional_anisotropy

from dipy.tracking import utils

import src.dwi_tools as dwi_tools
import src.nn_helper as nn_helper
import src.tracking as tracking

from dipy.tracking.streamline import values_from_volume
import dipy.align.vector_fields as vfu
from dipy.core.sphere import Sphere
from dipy.core import subdivide_octahedron 
from scipy.spatial import KDTree
from dipy.core import subdivide_octahedron 

from dipy.tracking.local import LocalTracking
from dipy.viz import window, actor
from dipy.viz.colormap import line_colors
from dipy.tracking.streamline import Streamlines, transform_streamlines
from dipy.tracking import metrics
from dipy.tracking.utils import random_seeds_from_mask, seeds_from_mask

import numpy as np
import warnings

import tensorflow as tf
from joblib import Parallel, delayed
import multiprocessing
from keras.models import load_model
from keras.layers import Activation

import importlib
importlib.reload(tracking)
importlib.reload(nn_helper)
import src.tracking as tracking
import tensorflow as tf
from src.nn_helper import swish, squared_cosine_proximity, squared_cosine_proximity_2

In [2]:
import os
print(os.getcwd())
os.chdir('..')
print(os.getcwd())

/home/nico/Code/deepFibreTracking/100307
/home/nico/Code/deepFibreTracking


# Multi (DWI,PrevTension) -> Tension tracker

In [350]:
pModel = 'results/102218_train_OLDsh4_step10_wholeBrain_b1k_1x1/models/mlp_doubleIn_single_sqCos2_wb_dx_1_dy_1_dz_1_dd_15_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_1_pt_0_4172--0.974801.h5'
pResult = pModel.replace('.h5','').replace('/models/','_') + '-prediction'

In [351]:
tracker = load_model(pModel , custom_objects={'tf':tf, 'swish':Activation(swish), 'squared_cosine_proximity': squared_cosine_proximity, 'squared_cosine_proximity_2': squared_cosine_proximity_2})
noSamples, noX, noY, noZ, noC = tracker.get_input_shape_at(0)[0]
tracker.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 1, 1, 15)  0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 15)           0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 3)            0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 18)           0           flatten_1[0][0]                  
                                                                 input_2[0][0]                    
__________

In [6]:
coordinateScaling = 1
useDTIPeakDirection = True
useSphericalHarmonics = False
print('Loaded model with  (dx %d, dy %d, dz %d) and %d channels' % (noX, noY, noZ, noC))

if(noC == 3):
    useDTIPeakDirection = True
    
if(noC == 15):
    useSphericalHarmonics = True
    sh_order = 4
elif(noC == 45):
    useSphericalHarmonics = True
    sh_order = 8

# load DWI data
print('Importing HCP data')
bvals,bvecs,gtab,dwi,aff,t1,binarymask = dwi_tools.loadHCPData('100307')
dwi_subset, gtab_subset, bvals_subset, bvecs_subset = dwi_tools.cropDatsetToBValue(1000, bvals, bvecs, dwi)
b0_idx = bvals < 10
b0 = dwi[..., b0_idx].mean(axis=3)

dwi_singleShell = np.concatenate((dwi_subset, dwi[..., b0_idx]), axis=3)
bvals_singleShell = np.concatenate((bvals_subset, bvals[..., b0_idx]), axis=0)
bvecs_singleShell = np.concatenate((bvecs_subset, bvecs[b0_idx,]), axis=0)
gtab_singleShell = gradient_table(bvals=bvals_singleShell, bvecs=bvecs_singleShell, b0_threshold = 10)

tracking_data = dwi_singleShell

### estimate tensors
#print('fitting single tensor model (OLS)')
#import dipy.reconst.dti as dti
#start_time = time.time()
#dti_wls = dti.TensorModel(gtab_subset, fit_method='LS')
#fit_wls = dti_wls.fit(dwi_subset)
#tensors = fit_wls.quadratic_form
#tensors = tensors.reshape( (tensors.shape[:-2] + (-1,) ) )
#runtime = time.time() - start_time
#print('-> runtime ' + str(runtime) + 's\n')

### exract the averaged b0.

if(useSphericalHarmonics):
    print('Spherical Harmonics')
    dwi_singleShell_norm = dwi_tools.normalize_dwi(dwi_singleShell, b0)
    
    csd_model = ConstrainedSphericalDeconvModel(gtab_singleShell, None, sh_order=sh_order)
    csd_fit = csd_model.fit(dwi_singleShell_norm, mask=binarymask)
    
    tracking_data = csd_fit.shm_coeff

## normalize DWI by b0
#print('\n normalizing dwi dataset')
#dwi_subset_normalized = dwi_tools.normalize_dwi(dwi_subset, b0)

# corpus callosum seed mask
ccmask, options = nrrd.read('100307/100307-ccSegmentation.nrrd')
ccseeds = seeds_from_mask(ccmask, affine=aff)
seeds_ras = ccseeds[45:48]

# load reference streamlines
streamlines_val = np.load('streamlines_validation.npy')

# project seeds into image coordinate system
#aff_ras_ijk = np.linalg.inv(aff) # aff: IJK -> RAS
#M = aff_ras_ijk[:3, :3]
#abc = aff_ras_ijk[:3, 3]
#abc = abc[:,None]
#seeds_ijk = (M.dot(seeds_ras.T) + abc).T


# DTI PEAK DIRECTION INPUT
if(useDTIPeakDirection):
    print('DTI Peak Direction/odf estimation')
    import dipy.reconst.dti as dti
    start_time = time.time()
    dti_model = dti.TensorModel(gtab_singleShell)#, fit_method='WLS')
    dti_fit = dti_model.fit(dwi_singleShell)
    dti_fit_odf = dti_fit.odf(sphere = default_sphere)
    runtime = time.time() - start_time
    print('Runtime ' + str(runtime) + 's')

    #TODO: not necessary in the future, replace with own function
    #sphere = get_sphere('symmetric724')
    #start_time = time.time()
    #dtipeaks = peaks_from_model(model=dti_model,
    #                    data=dwi_singleShell_norm,
    #                    sphere=sphere,
    #                    relative_peak_threshold=.5,
    #                    min_separation_angle=25,
    #                    mask=binarymask,
    #                    return_odf=False,
    #                    parallel=False,
    #                    normalize_peaks=False,
    #                    npeaks=5
    #                   )
    #runtime = time.time() - start_time
    #print('Runtime ' + str(runtime) + 's')
    #dtiPeakDirs = dtipeaks.peak_dirs[:,:,:,0,:]
    #tracking_data = dtiPeakDirs
    
# select appropriate dataset
#
#tracking_data = data_sh
#tracking_data = tensors

Loaded model with  (dx 1, dy 1, dz 1) and 15 channels
Importing HCP data
Spherical Harmonics
Percentage erroneous voxels: 4.07


/home/nico/Code/deepFibreTracking/src/dwi_tools.py:85: RuntimeWarning: divide by zero encountered in true_divide
  weights_normed = (weights / b0)
/home/nico/Code/deepFibreTracking/src/dwi_tools.py:85: RuntimeWarning: invalid value encountered in true_divide
  weights_normed = (weights / b0)


DTI Peak Direction/odf estimation
Runtime 827.2817482948303s


In [7]:
dwi_singleShell_norm = dwi_tools.normalize_dwi(dwi_singleShell, b0)
data_sh = dwi_tools.get_spherical_harmonics_coefficients(bvals=bvals_singleShell,bvecs=bvecs_singleShell,sh_order=4, dwi=dwi_singleShell_norm, b0 = 0)

Percentage erroneous voxels: 4.07


/home/nico/Code/deepFibreTracking/src/dwi_tools.py:85: RuntimeWarning: divide by zero encountered in true_divide
  weights_normed = (weights / b0)
/home/nico/Code/deepFibreTracking/src/dwi_tools.py:85: RuntimeWarning: invalid value encountered in true_divide
  weights_normed = (weights / b0)


In [358]:
wholebrainseeds = seeds_from_mask(binarymask, affine=aff)

In [25]:
seeds_ras = ccseeds
tracking_data = data_sh
stepWidth = 0.6

In [352]:
importlib.reload(tracking)
import src.tracking as tracking
# do tracking
start_time = time.time()
streamlines_mlp_simple,vNorms = tracking.start(bitracker=True, inverseDirection=False, seeds=seeds_ras,data=tracking_data, affine=aff, model=tracker, noX=noX, noY=noY, noZ=noZ, dw = noC, stepWidth = stepWidth, coordinateScaling = coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

start_time = time.time()
streamlines_mlp_simple_2,vNorms2 = tracking.start(bitracker=True, inverseDirection=True,seeds=seeds_ras,data=tracking_data, affine=aff, model=tracker, noX=noX, noY=noY, noZ=noZ, dw = noC, stepWidth = stepWidth, coordinateScaling = coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

# visualize results
streamlines_mlp_simple_f = np.fliplr(streamlines_mlp_simple)
streamlines_joined = np.concatenate([streamlines_mlp_simple_f,streamlines_mlp_simple_2],axis=1)    

streamlines_joined_imageCS = transform_streamlines(streamlines_joined, np.linalg.inv(aff))

np.save(pResult + '-ijk.vtk', streamlines_joined_imageCS)
np.save(pResult + '-ras.vtk', streamlines_joined)
dwi_tools.saveVTKstreamlines(streamlines_joined,pResult + '.vtk')

dwi_tools.visStreamlines(streamlines_joined_imageCS,t1, vol_slice_idx = 75)

/home/nico/Code/deepFibreTracking/src/tracking.py:161: RuntimeWarning: invalid value encountered in true_divide
  if(bitracker):
/home/nico/Code/deepFibreTracking/src/tracking.py:182: RuntimeWarning: invalid value encountered in double_scalars
  lv1 = predictedDirection[j,]
/home/nico/Code/deepFibreTracking/src/tracking.py:182: RuntimeWarning: invalid value encountered in arcsin
  lv1 = predictedDirection[j,]


Runtime 72.91352319717407 s 
Runtime 71.7975423336029 s 
0/1343
1000/1343
Wrote streamlines to results/102218_train_OLDsh4_step10_wholeBrain_b1k_1x1_mlp_doubleIn_single_sqCos2_wb_dx_1_dy_1_dz_1_dd_15_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_1_pt_0_4172--0.974801-prediction.vtk


/home/nico/anaconda3/lib/python3.6/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [76]:
from dipy.tracking.streamline import set_number_of_points, Streamlines
sl_est_2 = set_number_of_points(Streamlines(streamlines_joined_imageCS), nb_points = 20)
sl_val_2 = set_number_of_points(Streamlines(streamlines_val.tolist()), nb_points = 20)

In [77]:
from dipy.segment.metric import SumPointwiseEuclideanMetric
from dipy.segment import quickbundles

dist = quickbundles.bundles_distances_mdf(sl_est_2, sl_val_2)

In [97]:
dist_minIdx = np.argmin(dist, axis = 1)

---

# Single DWI -> TENSION tracker

In [353]:
#pModel = 'results/102218_train_OLDsh4_step10_wholeBrain_b1k_1x1/models/mlp_single_mse_wb_dx_1_dy_1_dz_1_dd_15_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_1_pt_0_950-0.489318.h5'

#pModel = 'results/102218_train_OLDsh4_step10_wholeBrain_b1k_1x1/models/mlp_single_cos_wb_dx_1_dy_1_dz_1_dd_15_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_1_pt_0_1173--0.263056.h5'

pModel = 'results/102218_train_OLDsh4_step10_wholeBrain_b1k_1x1/models/mlp_single_sqCos2_wb_dx_1_dy_1_dz_1_dd_15_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_1_pt_0_7217--0.968468.h5'

pModel = 'results/train_OLDsh4_step0.1_wholeBrain_b1k_1x1x1_step10/models/mlp_single_sqCos2_wb_dx_1_dy_1_dz_1_dd_15_ReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_1_bn_1_pt_0_6691--0.990052.h5'

pResult = pModel.replace('.h5','').replace('/models/','_') + '-prediction'

In [354]:
tracker = load_model(pModel , custom_objects={'tf':tf, 'swish':Activation(swish), 'squared_cosine_proximity': squared_cosine_proximity, 'squared_cosine_proximity_2': squared_cosine_proximity_2})
noSamples, noX, noY, noZ, noC = tracker.get_input_shape_at(0)
tracker.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 1, 1, 15)  0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 15)           0           input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 512)          8192        flatten_1[0][0]                  
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 512)          2048        dense_1[0][0]                    
__________________________________________________________________________________________________
re_lu_1 (R

In [ ]:
# do tracking (wholebrainseeds, ccseeds)
importlib.reload(tracking)
import src.tracking as tracking
start_time = time.time()
streamlines_mlp_simple, vec_norm = tracking.start(bitracker=False,inverseDirection=False,seeds=wholebrainseeds,data=tracking_data, affine=aff, model=tracker, noX=noX, noY=noY, noZ=noZ, dw = noC, stepWidth = 0.6, coordinateScaling = coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

start_time = time.time()
streamlines_mlp_simple_2, vec_norm_2 = tracking.start(bitracker=False,inverseDirection=True,seeds=wholebrainseeds,data=tracking_data, affine=aff, model=tracker, noX=noX, noY=noY, noZ=noZ, dw = noC, stepWidth = 0.6, coordinateScaling = coordinateScaling)
runtime = time.time() - start_time
print('Runtime ' + str(runtime) + ' s ')

# visualize results
streamlines_mlp_simple_f = np.fliplr(streamlines_mlp_simple)
streamlines_joined = np.concatenate([streamlines_mlp_simple_f,streamlines_mlp_simple_2],axis=1)    

streamlines_joined_imageCS = transform_streamlines(streamlines_joined, np.linalg.inv(aff))
                                                   
np.save(pResult + '-ijk.vtk', streamlines_joined_imageCS)
np.save(pResult + '-ras.vtk', streamlines_joined)
dwi_tools.saveVTKstreamlines(streamlines_joined,pResult + '.vtk')
                                                   
dwi_tools.visStreamlines(streamlines_joined_imageCS,t1, vol_slice_idx = 75)

/home/nico/Code/deepFibreTracking/src/tracking.py:161: RuntimeWarning: divide by zero encountered in true_divide
  if(bitracker):
/home/nico/Code/deepFibreTracking/src/tracking.py:161: RuntimeWarning: invalid value encountered in true_divide
  if(bitracker):
/home/nico/Code/deepFibreTracking/src/tracking.py:182: RuntimeWarning: invalid value encountered in double_scalars
  lv1 = predictedDirection[j,]
/home/nico/Code/deepFibreTracking/src/tracking.py:182: RuntimeWarning: invalid value encountered in arcsin
  lv1 = predictedDirection[j,]


In [357]:
dwi_tools.visTwoSetsOfStreamlines(transform_streamlines(streamlines_mlp_simple, np.linalg.inv(aff)),transform_streamlines(streamlines_mlp_simple_2, np.linalg.inv(aff)), t1, vol_slice_idx = 75)

/home/nico/anaconda3/lib/python3.6/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \
